In [68]:
from flask import Flask, request, flash, redirect, url_for, render_template, send_file, jsonify

import sys
import os
import datetime

import time

from modules.runner import Runner
   
runner = Runner()
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [69]:
# ------------------------------------- RUN APP -------------------------------------------------
app = Flask(__name__)
app.config['PATH_TO_ZIP'] = None

@app.route('/', methods=['GET'])
def upload_file():
    return render_template('initial.html')

@app.route('/', methods=['POST'])
def main_page():
    WORK_DIR = '../data/result_{}'.format(datetime.datetime.now().strftime('%Y%m%d%H%M%S'))
    os.makedirs(WORK_DIR)
    if 'file' not in request.files:
        flash('No file part')
        return redirect(request.url)
    file = request.files['file']
    # If the user does not select a file, the browser submits an
    # empty file without a filename.
    if file.filename == '':
        flash('No selected file')
        return redirect(request.url)
    if file:
        if allowed_file(file.filename):
            # если все ок, то сохраняем файл по директории UPLOAD_FOLDER
            filename = file.filename
            file_path = os.path.join(WORK_DIR, filename)
            file.save(file_path)
            # backend
            output_dir = os.path.join(WORK_DIR, os.path.basename(file_path).split('.')[0])
            os.mkdir(output_dir)
            time.sleep(3)
            results_path, path_to_zip = runner.process(file_path, output_dir, blur=True)
            app.config['PATH_TO_ZIP'] = path_to_zip
            return redirect('/processed/')
        else:
            flash('This file is not allowed')
            return redirect(request.url)
    return redirect(request.url)

@app.route('/wait/', methods=['GET'])
def upload_file_wait():
    return render_template('wait.html')
    
@app.route('/processed/', methods=['POST'])
def upload_file_post():
    try:
        return send_file(app.config['PATH_TO_ZIP'])
    except Exception as e:
        return str(e)
    
@app.route('/processed/', methods=['GET'])
def upload_file_get():
    return render_template('processed.html')


if __name__=='__main__': 
    #app.run(host='0.0.0.0', port='9889', debug=True)
    app.run(port="5000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Oct/2021 12:39:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 12:39:14] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [21/Oct/2021 12:39:14] "GET /processed/ HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2021 12:39:17] "POST /processed/ HTTP/1.1" 200 -
